Figures
* **Normalization:** Negcon normalization
* **mAP calculation:** mAP is calculated as difference to controls.

In [ ]:
### Modules import

from pathlib import Path

import pandas as pd
import plotly
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots

#### Reading the dataframes

In [ ]:
notebook_suffix = "48and49"
save_figures_externally = True

copairs_dir = Path("copairs_csv")
figures_dir = Path("figures")
if save_figures_externally:
    figures_dir.mkdir(exist_ok=True)

combined_moa_df = pd.read_csv(
    copairs_dir / "PrecisionValues_with_MoA_allplates_Negcon_wrt_Controls_48and49.csv"
)

### Comparison of Mean average precision

##### Comparison of Batch 3 (Cell Painting) and Batch 5 (ChromaLive Saguaro dyes)


In [ ]:
def quick_save(fig, fig_suffix: str, save_externally: bool = save_figures_externally):
    if save_externally:
        fig.write_html(str(figures_dir / f"{notebook_suffix}_{fig_suffix}.html"))
    else:
        fig.show("notebook")

In [ ]:
labels = {
    f"mean_average_precision_{batch}": name
    for batch, name in zip(
        ("batch3", "batch5"), ("mAP (Cell Painting)", "mAP (Saguaro)")
    )
}
first, second = labels.keys()
scatter = px.scatter(
    combined_moa_df,
    x=combined_moa_df[first],
    y=combined_moa_df[second],
    labels=labels,
    color=combined_moa_df["MoA"],
)
scatter.update_layout(legend=dict(orientation="h"), height=800, width=1000)
quick_save(scatter, "scatter_mAP_MoA")

### Mean average precision values of all compounds

In [ ]:
combined_box_plot = go.Figure()

for field, name in labels.items():
    combined_box_plot.add_trace(
        go.Box(
            y=combined_moa_df[field],
            name=name[5:-1],
            boxpoints="all",
            hovertext=combined_moa_df["MoA"] + "-" + combined_moa_df["Common Name"],
        )
    )

combined_box_plot.update_layout(
    height=800,
    width=1000,
    font_family="Arial",
    font=dict(size=14, color="Black"),
    boxmode="group",
    yaxis_title="Mean average precision",
)
# combined_box_plot.show("notebook")
quick_save(combined_box_plot, "boxplot_mAP_MoA")

In [ ]:
# tidy data frame first

from itertools import product

cols = ("mean_average_precision", "above_p_threshold", "above_q_threshold")
batches = ("batch3", "batch5")
# maps = combined_moa_df[[*map(lambda x: combined"_".join(x), product(cols, batches)), "BRD ID"]]


def combine_cols(df: pd.DataFrame, cols, batches, id_col: str = "BRD ID"):
    """
    Get a column-based data frame and make it denser
    """
    maps = []
    for col in cols:
        tmp = []
        for batch in batches:
            colname = "_".join((col, batch))
            df = df.loc(axis=1)[[colname, id_col]].copy()
            df.rename({colname: col}, inplace=True, axis=1)
            df["batch"] = batch
            tmp.append(df)

        maps.append(pd.concat(tmp, axis=0))
    return pd.concat(maps, axis=1)


result = combine_cols(combined_moa_df, cols, batches)

### Difference in mean average precision values

##### Standard Cell Painting
The negative values indicate the better performance of one set of dyes against the other

In [ ]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=combined_moa_df["MoA"],
        y=combined_moa_df["batch3_vs_batch5"],
        mode="markers",
        hovertext=combined_moa_df["Common Name"],
    )
)
fig.update_layout(
    height=1000, width=1700, font_family="Arial", font=dict(size=14, color="Black")
)
fig.update_yaxes(title="Difference in <br> Mean average precision")
fig.update_xaxes(categoryorder="total ascending")
# fig.show("notebook")

quick_save(fig, "scatter_MoA")